# Heart Disease Prediction using Random Forest

This notebook demonstrates the use of a Random Forest Classifier to predict heart disease risk using patient data. The workflow follows the CRISP-DM methodology.

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Load the heart disease dataset - Replace 'heart.csv' with the actual file path if needed.
heart_data = pd.read_csv('heart_dataset.csv')  

# Define features and target variable
X = heart_data.drop(columns=['target']) 
y = heart_data['target']  

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize the Random Forest classifier with adjusted hyperparameters
rf_model = RandomForestClassifier(random_state=42, 
                                  n_estimators=100,  # You can reduce this if still overfitting
                                  max_depth=5,       # Limit the depth of each tree
                                  min_samples_split=5, # Require more samples to split a node
                                  min_samples_leaf=2)  # Require more samples in a leaf node

# Fit the model to the training data
rf_model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = rf_model.predict(X_test)

# Evaluate the model's performance
accuracy = accuracy_score(y_test, y_pred)
classification_rep = classification_report(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)

accuracy, classification_rep, conf_matrix